# Starting with just 2023 for my laptop 

In [2]:
import duckdb
import pandas as pd
import pandas_ta as ta

con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")
print(con)

In [3]:
tables = con.execute('PRAGMA show_tables;').fetchall()
print(tables)

[('Stocks',)]


In [4]:
qry_year = '''
    SELECT * FROM Stocks
    WHERE EXTRACT(YEAR FROM Datetime) = 2023 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

qry = f'''
    SELECT * FROM Stocks
    WHERE Stock = {Stock} 
    AND Interval = 1
'''

df = con.execute(qry).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)


print(df.head())

             Datetime        open        high         low       close  volume  \
0 2023-01-03 04:00:00  129.391006  130.113998  129.386002  130.110001  5381.0   
1 2023-01-03 04:01:00  129.977005  130.283005  129.972000  130.209000  6072.0   
2 2023-01-03 04:02:00  130.285004  130.352005  130.270004  130.279007  1053.0   
3 2023-01-03 04:03:00  130.294998  130.401993  130.289993  130.388000  1754.0   
4 2023-01-03 04:04:00  130.384003  130.570999  130.378998  130.567001  3766.0   

  Stock  interval  
0  AAPL       1.0  
1  AAPL       1.0  
2  AAPL       1.0  
3  AAPL       1.0  
4  AAPL       1.0  


##  Creating and Flattening the Data for:
#### Packet Size (Only 1 min intervals)

### Getting the Data, and calculating P/L

In [5]:
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time']>start_time)&(df['Time']<end_time)]
print(TradingDay_df.info())

def trail_stoploss(index, tradingdata, whole_percentage):
    idx = index
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] # Get the dataframe from the row you want until 100 minutes later
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return [i, profit]
            
    




<class 'pandas.core.frame.DataFrame'>
Index: 104714 entries, 235 to 220254
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Datetime  104714 non-null  datetime64[us]
 1   open      104714 non-null  float32       
 2   high      104714 non-null  float32       
 3   low       104714 non-null  float32       
 4   close     104714 non-null  float32       
 5   volume    104714 non-null  float32       
 6   Stock     104714 non-null  object        
 7   interval  104714 non-null  float32       
 8   Time      104714 non-null  object        
dtypes: datetime64[us](1), float32(6), object(2)
memory usage: 5.6+ MB
None


In [6]:
print(TradingDay_df.index[0])

235


In [ ]:
import os
import csv

columns = []
flattened_df100 = []
columns100 = []

First_Index = TradingDay_df.index[0]
Index_Counter = First_Index

# Creating the CSV Files
# 100
packet = df[['open', 'close']].loc[First_Index-99:First_Index]
for i in range(len(packet)):
        columns100.append(f'open_{i+1}')
        columns100.append(f'close_{i+1}')
columns100.extend(["EMA100","EMA20","RSI","Profit","Index","Profit Index"])
with open('packet100.csv', 'a', newline='') as f:
      csv.writer(f).writerow(columns100)


for index, row in TradingDay_df.iterrows():
    
    if index % 5000 ==0:
         print(index)

    trail_stop= trail_stoploss(index, df, 5)
    profit = trail_stop[1]
    profit_index = trail_stop[0]
    

    # 100 Minutes
    packet100 = df[['open', 'close']].loc[index-99:index]
    flatpack100 = list(packet100.values.flatten())

    packet100['EMA100'] = ta.ema(packet100['close'], length=100)
    packet100['EMA20'] = ta.ema(packet100['close'], length=20)
    packet100['RSI'] = ta.rsi(packet100['close'])

    TA_100 = list(packet100[['EMA100','EMA20','RSI']].loc[index])
    flatpack100.extend(TA_100)
    flatpack100.extend([profit,index,profit_index])
    with open('packet100.csv', 'a', newline='') as f:
      csv.writer(f).writerow(flatpack100)

 
con.close() 
    


# Getting the Whole Dataset for every Stock 

In [6]:
con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")

qry = f'''
    SELECT DISTINCT(Stock) FROM Stocks
'''

df = con.execute(qry).fetchdf()
# Sort the Data from oldest to newest
print(list(df['Stock']))

['JNJ', 'GS', 'AMD', 'BAC', 'WMT', 'BA', 'KO', 'WST', 'SPY', 'NVDA', 'GE', 'AAPL', 'XOM', 'MCD', 'AMZN', 'PG']


In [8]:
def trail_stoploss(index, tradingdata, whole_percentage):
    idx = index
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] # Get the dataframe from the row you want until 100 minutes later
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return [i, profit]

In [10]:
import duckdb
import pandas as pd
import pandas_ta as ta
import os
import csv

con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")
Stocks = ['JNJ', 'GS', 'AMD', 'BAC', 'WMT', 'BA', 'KO', 'WST', 'SPY', 'NVDA', 'GE', 'AAPL', 'XOM', 'MCD', 'AMZN', 'PG']

for Stock in Stocks:
    print(Stock)

    qry = f'''
        SELECT * FROM Stocks
        WHERE Stock = '{Stock}' 
        AND Interval = 1
    '''

    df = con.execute(qry).fetchdf()
    # Sort the Data from oldest to newest
    df = df.sort_values(by='Datetime').reset_index(drop=True)

    df['Time'] = df['Datetime'].dt.time
    start_time = pd.to_datetime('09:00:00').time()
    end_time = pd.to_datetime('16:00:00').time()
    TradingDay_df = df[(df['Time']>start_time)&(df['Time']<end_time)]

    columns = []
    flattened_df100 = []
    columns100 = []

    First_Index = TradingDay_df.index[0]
    Index_Counter = First_Index

    # Creating the CSV Files
    # 100
    packet = df[['open', 'close']].loc[First_Index-99:First_Index]
    for i in range(len(packet)):
            columns100.append(f'open_{i+1}')
            columns100.append(f'close_{i+1}')
    columns100.extend(["EMA100","EMA20","RSI","Profit","Index","Profit Index"])
    with open(f"C:/Users/rybot/OneDrive/Databases/New100Packets/{Stock}_data.csv", 'a', newline='') as f:
        csv.writer(f).writerow(columns100)


    for index, row in TradingDay_df.iterrows():
        
        if index % 5000 ==0:
            print(index)

        trail_stop= trail_stoploss(index, df, 5)
        profit = trail_stop[1]
        profit_index = trail_stop[0]
        

        # 100 Minutes
        packet100 = df[['open', 'close']].loc[index-99:index]
        flatpack100 = list(packet100.values.flatten())

        packet100['EMA100'] = ta.ema(packet100['close'], length=100)
        packet100['EMA20'] = ta.ema(packet100['close'], length=20)
        packet100['RSI'] = ta.rsi(packet100['close'])

        TA_100 = list(packet100[['EMA100','EMA20','RSI']].loc[index])
        flatpack100.extend(TA_100)
        flatpack100.extend([profit,index,profit_index])
        with open(f"C:/Users/rybot/OneDrive/Databases/New100Packets/{Stock}_data.csv", 'a', newline='') as f:
            csv.writer(f).writerow(flatpack100)


con.close() 
    

JNJ
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
735000
740000
7

KeyboardInterrupt: 